In [42]:
import requests
import pandas as pd
import h3
from shapely.geometry import Point
from shapely import wkt
import geopandas as gpd


# Define the Overpass API endpoint and query
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area["name"="Chicago"]->.searchArea;
(
  node["amenity"="restaurant"](area.searchArea);
  node["amenity"="bar"](area.searchArea);
  node["amenity"="cafe"](area.searchArea);
  node["amenity"="hotel"](area.searchArea);
  node["amenity"="hospital"](area.searchArea);
  node["amenity"="airport"](area.searchArea);
  node["railway"="station"](area.searchArea);
  node["amenity"="public_transport"](area.searchArea);
);
out body;
"""

# Send the query to the Overpass API
response = requests.get(overpass_url, params={'data': overpass_query})
data = response.json()

In [2]:
# Extract relevant data from the response
pois = []
for element in data['elements']:
    tags = element.get('tags', {})
    poi_type = (tags.get('amenity') or tags.get('leisure') or 
                    tags.get('tourism') or tags.get('railway'))
    poi = {
        'id': element['id'],
        'lat': element['lat'],
        'lon': element['lon'],
        'name': tags.get('name'),
        "poi_type": poi_type
    }
    pois.append(poi)

# Create a DataFrame from the list of POIs
df = pd.DataFrame(pois)
df

,id,lat,lon,name,poi_type
0,252643581,42.008086,-87.665969,Morse,station
1,252643632,41.953887,-87.654928,Sheridan,station
2,252683058,41.947520,-87.653627,Addison,station
3,252683064,41.939796,-87.653328,Belmont,station
4,252683073,41.918242,-87.652631,Armitage,station
...,...,...,...,...,...
3603,12066835882,41.908816,-87.675250,CAVA,restaurant
3604,12067405402,41.956237,-87.708267,El Alebrije,restaurant
3605,12068333588,41.879842,-87.641360,Dylan's Tavern & Grill,bar
3606,12075629329,41.886611,-87.648314,Palilgu,restaurant


In [3]:
import seaborn as sns
df["poi_type"].value_counts()

poi_type
restaurant    1963
bar            758
cafe           701
station        186
Name: count, dtype: int64

In [4]:
def add_h3_location(row, resolution):
    return h3.geo_to_h3(row['lat'], row['lon'], resolution)

resolutions = [4,6,8]

# H3-Location-Spalte hinzufügen
for res in resolutions:
    df[f"h3_res_{res}"] = df.apply(lambda row: add_h3_location(row, res), axis=1)
df

,id,lat,lon,name,poi_type,h3_res_4,h3_res_6,h3_res_8
0,252643581,42.008086,-87.665969,Morse,station,842664dffffffff,862664d8fffffff,882664d8c1fffff
1,252643632,41.953887,-87.654928,Sheridan,station,842664dffffffff,862664c17ffffff,882664c169fffff
2,252683058,41.947520,-87.653627,Addison,station,842664dffffffff,862664c17ffffff,882664c16bfffff
3,252683064,41.939796,-87.653328,Belmont,station,842664dffffffff,862664c17ffffff,882664c163fffff
4,252683073,41.918242,-87.652631,Armitage,station,842664dffffffff,862664c17ffffff,882664c13dfffff
...,...,...,...,...,...,...,...,...
3603,12066835882,41.908816,-87.675250,CAVA,restaurant,842664dffffffff,862664cafffffff,882664cac1fffff
3604,12067405402,41.956237,-87.708267,El Alebrije,restaurant,842664dffffffff,862664ca7ffffff,882664ca41fffff
3605,12068333588,41.879842,-87.641360,Dylan's Tavern & Grill,bar,842664dffffffff,862664c1fffffff,882664c1adfffff
3606,12075629329,41.886611,-87.648314,Palilgu,restaurant,842664dffffffff,862664cafffffff,882664cad3fffff


# Add censustract

In [5]:
df['census_tract'] = '1'

In [19]:
census_tract_data = gpd.read_file(r'../data/Census_tract/tl_2018_17_tract.shx')

In [79]:
def check_polygon(value):
    for i in range(len(census_tract_data)):
        if value.within(census_tract_data.iloc[i]['geometry']):  
            return census_tract_data.iloc[i]['GEOID']
            break

def insert_census_shape(x, census_tract, longitude, latitude):
    new_point = Point(x[longitude], x[latitude])
    x[census_tract] = check_polygon(new_point)
    
    return x[[census_tract]]

def map_back_tcd(value,census_tract):
    df.loc[value.name, census_tract] = value[0]

In [82]:
def insert_census_tract_data(census_tract, longitude, latitude):
    columns1 = df[[longitude, latitude, census_tract]]
    result = columns1.apply(
        insert_census_shape, axis=1, census_tract=census_tract, longitude=longitude, latitude=latitude)
    result.name = census_tract
    result.apply(map_back_tcd, axis = 1, census_tract=census_tract);

In [83]:
insert_census_tract_data('census_tract', 'lon', 'lat')

  census_tract
0  17031010600
1  17031832100
2  17031061000
3  17031063000
4  17031071100


In [84]:
df.head()

,id,lat,lon,name,poi_type,h3_res_4,h3_res_6,h3_res_8,census_tract
0,252643581,42.008086,-87.665969,Morse,station,842664dffffffff,862664d8fffffff,882664d8c1fffff,17031010600
1,252643632,41.953887,-87.654928,Sheridan,station,842664dffffffff,862664c17ffffff,882664c169fffff,17031832100
2,252683058,41.947520,-87.653627,Addison,station,842664dffffffff,862664c17ffffff,882664c16bfffff,17031061000
3,252683064,41.939796,-87.653328,Belmont,station,842664dffffffff,862664c17ffffff,882664c163fffff,17031063000
4,252683073,41.918242,-87.652631,Armitage,station,842664dffffffff,862664c17ffffff,882664c13dfffff,17031071100


In [85]:
df.to_csv("poi_data.csv",index=False)